In [1]:
import os
import json
import pandas as pd
import traceback

In [7]:
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv

In [3]:
key = os.getenv("OPENAI_API_KEY")

In [4]:
print(key)

sk-proj-sNiKBrmregdyYDknOHAFT3BlbkFJ1a9RWOW70w8IuZtDoP4v


In [5]:
llm = ChatOpenAI(openai_api_key=key, temperature=0.7, model_name="gpt-3.5-turbo")

/opt/anaconda3/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
